## BUS 243 Mini Project 1: Analysis of Decline in User Engagement at eLinks

The objective of this study is to examine and quantify the recent drop in engagement of users on eLinks. Additionally, this study aims to develop and test different hypotheses for a decline in engagement and make recommendations for addressing the decline in engagement.

In [3]:
from pyspark.sql.types import *

### Data Preparation

#### Loading Users Data and Creating a Temporary Table

In [6]:
# File location and type
file_location = "/FileStore/tables/users.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_users = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [7]:
# Create a user data table
temp_table_name = "users_csv"
df_users.createOrReplaceTempView(temp_table_name)

#### Loading Emails Data and Creating a Temporary Table

In [9]:
# File location and type
file_location = "/FileStore/tables/emails.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_emails = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [10]:
# Create an emails data table
temp_table_name = "emails_csv"
df_emails.createOrReplaceTempView(temp_table_name)

#### Loading Events Data and Creating a Temporary Table

In [12]:
# File location and type
file_location = "/FileStore/tables/events.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_events = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [13]:
# Create an events table
temp_table_name = "events_csv"
df_events.createOrReplaceTempView(temp_table_name)

### Viewing the Data

#### Users Data

In [16]:
%sql 
select 
cast(user_id as int),
cast(created_at as timestamp), 
cast(company_id as int), 
language, 
cast(activated_at as timestamp),
state
from users_csv limit 5

user_id,created_at,company_id,language,activated_at,state
0,2013-01-01T20:59:39.000+0000,5737,english,2013-01-01T21:01:07.000+0000,active
1,2013-01-01T13:07:46.000+0000,28,english,null,pending
2,2013-01-01T10:59:05.000+0000,51,english,null,pending
3,2013-01-01T18:40:36.000+0000,2800,german,2013-01-01T18:42:02.000+0000,active
4,2013-01-01T14:37:51.000+0000,5110,indian,2013-01-01T14:39:05.000+0000,active


#### Emails Data

In [18]:
%sql 
select 
cast(user_id as int), 
cast(occurred_at as timestamp), 
action, 
cast(user_type as int)
from emails_csv limit 5

user_id,occurred_at,action,user_type
0,2014-05-06T09:30:00.000+0000,sent_weekly_digest,1
0,2014-05-13T09:30:00.000+0000,sent_weekly_digest,1
0,2014-05-20T09:30:00.000+0000,sent_weekly_digest,1
0,2014-05-27T09:30:00.000+0000,sent_weekly_digest,1
0,2014-06-03T09:30:00.000+0000,sent_weekly_digest,1


#### Events Data

In [20]:
%sql 
select 
cast(user_id as int), 
cast(occurred_at as timestamp), 
event_type, 
event_name, 
location, 
device, 
cast(user_type as int)
from events_csv limit 5

user_id,occurred_at,event_type,event_name,location,device,user_type
10522,2014-05-02T11:02:39.000+0000,engagement,login,Japan,dell inspiron notebook,3
10522,2014-05-02T11:02:53.000+0000,engagement,home_page,Japan,dell inspiron notebook,3
10522,2014-05-02T11:03:28.000+0000,engagement,like_message,Japan,dell inspiron notebook,3
10522,2014-05-02T11:04:09.000+0000,engagement,view_inbox,Japan,dell inspiron notebook,3
10522,2014-05-02T11:03:16.000+0000,engagement,search_run,Japan,dell inspiron notebook,3


### Data Exploration

#### Total Number of Engagements Per Day

The graph below shows the total number of engagement activities per day for the entire dataset. As shown by the graph, there is a moderate decline in the number of daily engagements starting at the end of July. To get a better idea of the trend of the data, we can also examine the total number of weekly engagements.

In [24]:
%sql 
SELECT 
DATE_TRUNC('day',occurred_at) AS day, 
COUNT(user_id) AS total_number_engagements
FROM events_csv 
GROUP BY day, event_type
HAVING event_type='engagement'

day,total_number_engagements
2014-07-25T00:00:00.000+0000,1516
2014-06-07T00:00:00.000+0000,519
2014-05-07T00:00:00.000+0000,1361
2014-07-06T00:00:00.000+0000,216
2014-07-04T00:00:00.000+0000,1657
2014-07-17T00:00:00.000+0000,1265
2014-06-21T00:00:00.000+0000,637
2014-07-30T00:00:00.000+0000,1498
2014-05-26T00:00:00.000+0000,1129
2014-08-23T00:00:00.000+0000,417


#### Total Number of Engagements Per Week

When we look at the total number of engagements per week, we can see a clear drop in engagements starting at the end of July (July 27th). For the week of July 27th, the number of weekly engagements was 8,291, and for the week of August 24th it was 6,038. This is a weekly decline in engagement activity of over 2000. This suggests that an event that led to a decline in engagement may have occurred at the end of July. We should also examine the user data to see if we see a similar decline in the number of registered users in the same time period.

In [27]:
%sql 
SELECT 
DATE_TRUNC('week',occurred_at) AS week, 
COUNT(user_id) AS total_number_engagements
FROM events_csv 
GROUP BY week, event_type
HAVING event_type='engagement'

week,total_number_engagements
2014-05-26T00:00:00.000+0000,7590
2014-08-11T00:00:00.000+0000,5922
2014-08-04T00:00:00.000+0000,6425
2014-07-28T00:00:00.000+0000,8291
2014-06-09T00:00:00.000+0000,7518
2014-07-14T00:00:00.000+0000,8070
2014-05-05T00:00:00.000+0000,7787
2014-05-12T00:00:00.000+0000,7979
2014-06-02T00:00:00.000+0000,8133
2014-05-19T00:00:00.000+0000,7831


#### All Users and Activated Users

The chart below shows the total number of all users and activated users per week. The graph shows that there is a dip in both the number of activated and all users between July 27th and August 10th, however there is overall positive growth in the number of both activate and all users. We would expect the number of users to be higher than the number of activated users, as users may sign up for eLinks and later change their mind, or forget to activate their account. Interestingly, people signed up for eLinks starting in January 2013, however the first engagement activity didn't take place until May 1st 2014, over a year after users started signing up for eLinks. This could be due to the critical mass of users required for social media networks to operate effectively.

In [30]:
%sql
SELECT DATE_TRUNC('week',created_at) AS week, COUNT(*) AS all_users,
	COUNT(CASE WHEN activated_at IS NOT NULL THEN u.user_id ELSE NULL END) AS activated_users
FROM `users_csv` u GROUP BY 1 ORDER BY 1

week,all_users,activated_users
2012-12-31T00:00:00.000+0000,56,26
2013-01-07T00:00:00.000+0000,67,29
2013-01-14T00:00:00.000+0000,77,47
2013-01-21T00:00:00.000+0000,77,36
2013-01-28T00:00:00.000+0000,75,30
2013-02-04T00:00:00.000+0000,87,48
2013-02-11T00:00:00.000+0000,81,41
2013-02-18T00:00:00.000+0000,82,39
2013-02-25T00:00:00.000+0000,81,33
2013-03-04T00:00:00.000+0000,85,43


As shown below, the first engagement activity took place on May 1st 2014, and the last engagement activity took place at the end of August.

In [32]:
%sql 
SELECT 
MIN(occurred_at) AS first_engagement,
MAX(occurred_at) AS last_engagement
FROM events_csv
WHERE event_type='engagement'

first_engagement,last_engagement
2014-05-01 02:27:15,2014-08-31 20:23:22


### Hypotheses

The following Hypotheses will be tested: 
1. **Hypothesis 1:** Users Who Joined eLinks a Long Time Ago Are No Longer Engaging
2. **Hypothesis 2:** Certain Elements of the Website Were Non-Functional, Resulting in a Decline in Engagement
3. **Hypothesis 3:** Some Users Have Switched Over to a Different Platform
4. **Hypothesis 4:** eLinks Changed Their Engagement Techniques

#### Hypothesis 1: Users Who Joined eLinks a Long Time Ago Are No Longer Engaging

The first hypothesis to test is that older users are no longer engaging with the platform, which is why the engagement rate has declined recently. This hypothesis was developed based on the logic that some users who joined the platform a long time ago may have forgotten that they ever joined the platform, or after using the platform for some time decided to switch to a different platform if eLinks didn’t meet their expectations.

In [37]:
%sql 

SELECT 
day_of_activation,
last_engagement_for_user,
last_engagement,
CASE 
WHEN last_engagement_for_user IS NULL THEN DATEDIFF(last_engagement,day_of_activation) 
ELSE DATEDIFF(last_engagement,last_engagement_for_user) 
END AS days_since_last_engagement,
DATEDIFF(last_engagement,day_of_activation) AS days_on_platform

FROM(
SELECT 
CAST(u.user_id AS int), 
DATE_TRUNC('day',u.activated_at) AS day_of_activation, 
MAX(DATE_TRUNC('day',e.occurred_at)) AS last_engagement_for_user, 
(SELECT 
MAX(DATE_TRUNC('day',occurred_at)) AS last_engagement
FROM events_csv
WHERE event_type='engagement') AS last_engagement
FROM users_csv u LEFT JOIN events_csv e ON 
u.user_id = e.user_id
GROUP BY u.user_id, u.activated_at
HAVING u.activated_at IS NOT NULL
ORDER BY u.user_id ASC)

day_of_activation,last_engagement_for_user,last_engagement,days_since_last_engagement,days_on_platform
2013-01-01T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,607,607
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160
2014-03-24T00:00:00.000+0000,null,2014-08-31T00:00:00.000+0000,160,160


As shown by the scatter plot above, there does not appear to be a relationship between the number of days a user has been on the platform and how recently they last engaged on eLinks. Based on the data shown above, there appear to be three main clusters of users:


  1. **New users with moderate activity:**
        Users who are new to the platform (less than 150 days) and engaged 100-150 days ago 
  2. **Long-term users with high-moderate activity:**
        Users who have been on the platform a long time (500-550 days) and have engaged within the last 100 days 
  3. **Long-term users who have been inactive for a long time:**
        Users who have been on the platform a long time (500-600 days) and last engaged 500-600 days ago

#### Hypothesis 2: Certain Elements of the Website Were Non-Functional, Resulting in a Decline in Engagement

The second hypothesis is that certain elements of the website were non-functional, resulting in a decline in engagement. For example, if users were not able to “like” posts by other users due to a bug on the website, this would result in a decline in engagement activities. While this cannot be tested directly, we can examine the number of different engagement activities and look for a sharp decline in one or more of them.

In [41]:
%sql 
SELECT 
DATE_TRUNC('week',occurred_at) AS week, 
COUNT(user_id) AS total_number_engagements,
event_name
FROM events_csv 
GROUP BY week, event_type, event_name
HAVING event_type='engagement'

week,total_number_engagements,event_name
2014-07-14T00:00:00.000+0000,16,search_click_result_10
2014-07-14T00:00:00.000+0000,24,search_click_result_3
2014-07-14T00:00:00.000+0000,15,search_click_result_7
2014-08-25T00:00:00.000+0000,382,search_autocomplete
2014-06-09T00:00:00.000+0000,891,login
2014-06-09T00:00:00.000+0000,1322,view_inbox
2014-06-16T00:00:00.000+0000,43,search_click_result_2
2014-07-28T00:00:00.000+0000,31,search_click_result_3
2014-08-25T00:00:00.000+0000,1108,view_inbox
2014-06-02T00:00:00.000+0000,2412,home_page


As shown by the line graph above, the engagement activity with the highest decline is users loading the home page of eLinks. This does not support the hypothesis that certain features of the website were not operational, as it shows a decline in the number of people loading the website itself. There may have be an external factor that caused a decline in the number of people vising eLinks.

To make sure that the website was fully functional at the end of August, we can also look at the number of signup activities that took place in that time period. The graph below shows the number of signup activities per day. As shown, the number of sign up activities follows a cyclical pattern, with certain times of the week being more popular to sign up for eLinks. The graph shows usual sign up activity during the end of August, so we can conclude that the website was functioning normally.

In [44]:
%sql
SELECT 
CAST(DATE_TRUNC('day',occurred_at) AS timestamp) AS day,
COUNT(user_id) AS number_signup_activities
FROM events_csv
WHERE event_type ='signup_flow'
GROUP BY day
ORDER BY day ASC

day,number_signup_activities
2014-05-01T00:00:00.000+0000,187
2014-05-02T00:00:00.000+0000,151
2014-05-03T00:00:00.000+0000,47
2014-05-04T00:00:00.000+0000,55
2014-05-05T00:00:00.000+0000,140
2014-05-06T00:00:00.000+0000,147
2014-05-07T00:00:00.000+0000,172
2014-05-08T00:00:00.000+0000,176
2014-05-09T00:00:00.000+0000,165
2014-05-10T00:00:00.000+0000,46


#### Hypothesis 3: Some Users Have Switched Over to a Different Platform

The third hypothesis is that there is a decline in engagement due to emergence of a competitor networking site. We can look for evidence for this hypothesis by examining whether there is a substantial decline in activity from users from a certain region or company, as new websites typically become popular regionally before spreading to other areas, and some companies encourage their employees to use a specific networking site. If your company recommends that you use a certain site and all your colleagues are using that site, it is likely that you would switch over to that site.

In [47]:
%sql

SELECT 
CAST(user_id AS int),
DATE_TRUNC('week',occurred_at) AS week, 
COUNT(user_id) AS total_number_engagements, 
location
FROM events_csv  
GROUP BY week, event_type, user_id, location
HAVING event_type='engagement'

user_id,week,total_number_engagements,location
12350,2014-05-12T00:00:00.000+0000,3,Australia
12402,2014-05-12T00:00:00.000+0000,2,Italy
12492,2014-05-12T00:00:00.000+0000,9,United States
12589,2014-06-02T00:00:00.000+0000,3,Canada
12689,2014-05-19T00:00:00.000+0000,2,Sweden
13046,2014-05-26T00:00:00.000+0000,3,Japan
13267,2014-05-26T00:00:00.000+0000,6,United Kingdom
13765,2014-06-02T00:00:00.000+0000,3,United States
13979,2014-06-09T00:00:00.000+0000,15,Poland
14277,2014-06-16T00:00:00.000+0000,9,Russia


As shown by the line graph above, Germany is the country with the largest number of engagements. There is a significant drop in the number of weekly engagements from 694 in the week of July 20th, 2014 to 355 in the week of August 24th, 2014. Other than May, the July value is the lowest that the weekly number of engagements has ever been in Germany. We don't consider the May value to be the low point because this is the start of engagement activity across the dataset.

France is the country with the second highest number of engagements. As shown by the graph, there is gradual decline from 496 weekly engagements in the week of June 29th to 274 weekly engagements in the week of August 24th. As Germany and France are geographically close, and both have a significant decline in engagement activity, it's possible that a competitor networking website has emerged in this region.

Further analysis is required to determine the cause for the significant drop in engagement of users located in Germany and France. Next, we can look at the engagement activitiy for all countries except Germany and France, to see if the decline in the engagement in these countries accounts for the decline in the overall weekly engagement.

In [51]:
%sql
SELECT 
CAST(user_id AS int),
DATE_TRUNC('week',occurred_at) AS week, 
COUNT(user_id) AS total_number_engagements, 
location
FROM events_csv
WHERE location NOT IN ('Germany','France')
GROUP BY week, event_type, user_id, location
HAVING event_type='engagement'

user_id,week,total_number_engagements,location
12350,2014-05-12T00:00:00.000+0000,3,Australia
12402,2014-05-12T00:00:00.000+0000,2,Italy
12492,2014-05-12T00:00:00.000+0000,9,United States
12589,2014-06-02T00:00:00.000+0000,3,Canada
12689,2014-05-19T00:00:00.000+0000,2,Sweden
13046,2014-05-26T00:00:00.000+0000,3,Japan
13267,2014-05-26T00:00:00.000+0000,6,United Kingdom
13765,2014-06-02T00:00:00.000+0000,3,United States
13979,2014-06-09T00:00:00.000+0000,15,Poland
14277,2014-06-16T00:00:00.000+0000,9,Russia


As shown by the line graph above, if we do not include the weekly engagement data from Germany and France, there is still a significant decline in the number of weekly engagements from the week of July 27th to August 24th. Therefore, the decline in engagement in Germany and France alone cannot account for the significant decrease in engagement.

Next, we can look at whether there is a sharp decline in engagement from users from specific companies to see if there may be a large company that changed their social media strategy and started encouraging their employees to use a different networking platform. If the company is large enough, such a change in strategy could negatively impact overall engagement on eLinks.

In [54]:
%sql 
SELECT 
week, 
COUNT(user_id) AS weekly_engagements, 
CAST(company_id AS int)

FROM

(SELECT
e.user_id, 
DATE_TRUNC('week',e.occurred_at) AS week, 
u.company_id
FROM events_csv e LEFT JOIN users_csv u
  ON e.user_id = u.user_id
WHERE e.event_type='engagement')

GROUP BY week, company_id

week,weekly_engagements,company_id
2014-05-05T00:00:00.000+0000,10,10544
2014-05-05T00:00:00.000+0000,5,68
2014-06-02T00:00:00.000+0000,5,7130
2014-06-09T00:00:00.000+0000,5,10
2014-06-09T00:00:00.000+0000,5,7467
2014-06-16T00:00:00.000+0000,62,4
2014-06-23T00:00:00.000+0000,4,7407
2014-07-14T00:00:00.000+0000,5,3194
2014-07-21T00:00:00.000+0000,23,13173
2014-07-28T00:00:00.000+0000,20,5650


As shown by the line chart above, the company with ID "1" has the highest number of weekly engagements. There has been a significant drop in engagements from 512 weekly engagements for the week of July 27th to 176 weekly engagements for the week of August 24th. Given that this company was responsible for most of the weekly engagements, it's possible that a decline in engagement from one company could have an impact on the overall engagement on eLinks. The company with ID "2" has also had a significant decline in engagement recently, from 341 weekly engagements for the week of July 20th, to a low of  47 engagements for the week of August 17th. As mentioned previously, possible reasons for the decline in engagement of specific groups could be due to the emergence of a competitor platform.

The data presented in this section supports the idea that there has been a decline in the engagement of the companies and locations that previously had the highest engagement rates. This supports the hypothesis that some people may have started using a different networking site, however further evidence is required to confirm this hypothesis.

#### Hypothesis 4: eLinks Changed Their Engagement Techniques

The final hypothesis is that eLinks changed their engagement techniques, perhaps by reducing the number of weekly digest emails sent to their users. To assess this hypothesis, we will first look at the number of weekly emails sent.

As shown by the graph below, there is a steady increase in the number of weekly digest emails sent per day. The highest number of emails are sent out on Mondays, with lower numbers of emails being sent out on Tuesday, Wednesday, Thursday and Friday.

In [60]:
%sql 
SELECT 
COUNT(CAST(user_id AS int)) AS weekly_emails_sent, 
DATE_TRUNC('day',CAST(occurred_at AS timestamp)) AS day
FROM emails_csv
WHERE action='sent_weekly_digest'
GROUP BY day
ORDER BY day ASC

weekly_emails_sent,day
467,2014-05-01T00:00:00.000+0000
441,2014-05-02T00:00:00.000+0000
760,2014-05-05T00:00:00.000+0000
477,2014-05-06T00:00:00.000+0000
435,2014-05-07T00:00:00.000+0000
479,2014-05-08T00:00:00.000+0000
451,2014-05-09T00:00:00.000+0000
766,2014-05-12T00:00:00.000+0000
491,2014-05-13T00:00:00.000+0000
447,2014-05-14T00:00:00.000+0000


If we consider the same data at the week level instead of the day level, we can see that there is a gradual increase in the number of weekly digest emails sent out each week, as shown by the bar chart below, which makes sense given the increase in users over time.

In [62]:
%sql 
SELECT 
COUNT(CAST(user_id AS int)) AS weekly_digest_emails_sent, 
DATE_TRUNC('week',CAST(occurred_at AS timestamp)) AS week
FROM emails_csv
WHERE action='sent_weekly_digest'
GROUP BY week
ORDER BY week ASC

weekly_digest_emails_sent,week
908,2014-04-28T00:00:00.000+0000
2602,2014-05-05T00:00:00.000+0000
2665,2014-05-12T00:00:00.000+0000
2733,2014-05-19T00:00:00.000+0000
2822,2014-05-26T00:00:00.000+0000
2911,2014-06-02T00:00:00.000+0000
3003,2014-06-09T00:00:00.000+0000
3105,2014-06-16T00:00:00.000+0000
3207,2014-06-23T00:00:00.000+0000
3302,2014-06-30T00:00:00.000+0000


Next, we can consider the number of reengagement emails sent out. Reengagement emails are presumably sent out to users who haven't logged on to eLinks for a while in an attempt to re-engage them. As shown by the bar chart below, there number of reengagement emails increased slightly in the last two weeks of the dataset.

In [64]:
%sql 
SELECT 
COUNT(CAST(user_id AS int)) AS reengagement_emails_sent, 
DATE_TRUNC('week',CAST(occurred_at AS timestamp)) AS week
FROM emails_csv
WHERE action='sent_reengagement_email'
GROUP BY week
ORDER BY week ASC

reengagement_emails_sent,week
98,2014-04-28T00:00:00.000+0000
164,2014-05-05T00:00:00.000+0000
175,2014-05-12T00:00:00.000+0000
179,2014-05-19T00:00:00.000+0000
179,2014-05-26T00:00:00.000+0000
199,2014-06-02T00:00:00.000+0000
190,2014-06-09T00:00:00.000+0000
234,2014-06-16T00:00:00.000+0000
187,2014-06-23T00:00:00.000+0000
222,2014-06-30T00:00:00.000+0000


The data explored in this section does not support the hypothesis that the recent decline in engagement is due to a change in engagement tactics as the data shows a slight increase in the number of engagement and re-engagement emails sent out.

### Analysis of Engagement Strategies

To help eLinks determine how effective their weekly digest emails are at getting their users to engage with their website, we can compare the engagement rate of users who clicked through and did not click through their emails from eLinks. In addition to the analysis in the hypothesis section, this analysis of engagement techniques will inform the recommendations for eLinks.

#### Daily Engagement for Users Who Did Not Click The Weekly Digest Email

The chart below shows the distribution of daily engagement data for all users who did not click through the email on a given day. As shown, the median number of engagements is 8 and there is a large spread in the data with a high number of outliers with a high daily engagement.

In [70]:
%sql 

SELECT 
e.ID_day_engagement,
ct.ID_day_email_clickthrough,
nvl(e.engagements,0) AS user_engagements_day,
nvl(ct.clickthroughs,0) AS user_clickthroughs_day

FROM 
(SELECT 
CONCAT(CAST(user_id AS int),'|',DATE_TRUNC('day',CAST(occurred_at AS timestamp))) AS ID_day_engagement,
COUNT(CAST(user_id AS int)) AS engagements
FROM events_csv 
WHERE event_type = 'engagement'
GROUP BY ID_day_engagement) e

FULL OUTER JOIN (SELECT 
CONCAT(CAST(user_id AS int),'|',DATE_TRUNC('day',CAST(occurred_at AS timestamp))) AS ID_day_email_clickthrough,
COUNT(CAST(user_id AS int)) AS clickthroughs
FROM emails_csv
WHERE action = 'email_clickthrough'
GROUP BY ID_day_email_clickthrough) ct

ON e.ID_day_engagement=ct.ID_day_email_clickthrough

WHERE nvl(ct.clickthroughs,0)=0

ID_day_engagement,ID_day_email_clickthrough,user_engagements_day,user_clickthroughs_day
1078|2014-07-25 00:00:00,null,8,0
11536|2014-05-05 00:00:00,null,19,0
1170|2014-07-25 00:00:00,null,5,0
12397|2014-05-23 00:00:00,null,8,0
12478|2014-05-14 00:00:00,null,6,0
1283|2014-08-22 00:00:00,null,9,0
12853|2014-06-04 00:00:00,null,15,0
1287|2014-05-21 00:00:00,null,12,0
12912|2014-05-22 00:00:00,null,3,0
1308|2014-08-21 00:00:00,null,8,0


#### Daily Engagement for Users Who Did Click Through the Weekly Digest Email

Next, we can consider the daily engagement of all users who did click through the email they received on a given day. Interestingly, the daily engagement of those users who did click through the weekly digest email is lower than the distribution of the daily engagement of users who did not click through the weekly digest email, with a median of 3 daily engagements. The upper and lower quartile bounds of the distribution of the daily engagement data for users who did click on the email is much smaller than the data for users who did not click through the email. Similar to the distribution of the data for users who did not click through the email, there are a large number of outliers with a high daily engagement rate, with the highest daily engagement rate being 65 engagement activities for a given day.

In [73]:
%sql 

SELECT 
e.ID_day_engagement,
ct.ID_day_email_clickthrough,
nvl(e.engagements,0) AS user_engagements_day,
nvl(ct.clickthroughs,0) AS user_clickthroughs_day

FROM 
(SELECT 
CONCAT(CAST(user_id AS int),'|',DATE_TRUNC('day',CAST(occurred_at AS timestamp))) AS ID_day_engagement,
COUNT(CAST(user_id AS int)) AS engagements
FROM events_csv 
WHERE event_type = 'engagement'
GROUP BY ID_day_engagement) e

FULL OUTER JOIN (SELECT 
CONCAT(CAST(user_id AS int),'|',DATE_TRUNC('day',CAST(occurred_at AS timestamp))) AS ID_day_email_clickthrough,
COUNT(CAST(user_id AS int)) AS clickthroughs
FROM emails_csv
WHERE action = 'email_clickthrough'
GROUP BY ID_day_email_clickthrough) ct

ON e.ID_day_engagement=ct.ID_day_email_clickthrough

WHERE nvl(ct.clickthroughs,0)>=1

ID_day_engagement,ID_day_email_clickthrough,user_engagements_day,user_clickthroughs_day
655|2014-07-31 00:00:00,655|2014-07-31 00:00:00,8,1
2050|2014-07-25 00:00:00,2050|2014-07-25 00:00:00,29,1
4032|2014-06-23 00:00:00,4032|2014-06-23 00:00:00,12,1
4163|2014-05-05 00:00:00,4163|2014-05-05 00:00:00,5,1
5508|2014-07-09 00:00:00,5508|2014-07-09 00:00:00,13,1
5597|2014-08-15 00:00:00,5597|2014-08-15 00:00:00,4,1
5633|2014-06-02 00:00:00,5633|2014-06-02 00:00:00,29,1
6074|2014-06-10 00:00:00,6074|2014-06-10 00:00:00,11,1
null,6401|2014-07-04 00:00:00,0,1
null,7089|2014-05-16 00:00:00,0,1


The data presented above shows an unexpected trend, that on a given day users who clicked through the weekly digest email that day tend to have a lower engagement on that day compared with the data for users that did not click through the weekly digest email. To examine the effectiveness of the weekly digest email at a high level, we can also consider the data at the user level to see if there is any correlation between the number of times a user has clicked through the weekly digest email and the number of times they have engaged on the platform.

The scatterplot below shows the relationship shows the relationship between the total number of clickthroughs and total number of engagements actions at the user level. As shown, there does not appear to be a strong relationship between the total clickthroughs and total engagement actions per user, which suggests that sending the weekly digest emails may not be the most effective engagement strategy that eLinks could use.

In [76]:
%sql 

SELECT 
e.user_id, 
ct.user_id,
NVL(ct.total_clickthroughs,0) AS total_clickthroughs,
NVL(e.total_engagements,0) AS total_engagements

FROM (SELECT 
CAST(user_id AS int),
COUNT(CAST(user_id AS int)) AS total_engagements
FROM events_csv 
WHERE event_type = 'engagement'
GROUP BY user_id) e 

FULL OUTER JOIN 

(SELECT 
CAST(user_id AS int), 
COUNT(CAST(user_id AS int)) AS total_clickthroughs
FROM emails_csv
WHERE action = 'email_clickthrough'
GROUP BY user_id) ct

ON e.user_id=ct.user_id

user_id,user_id,total_clickthroughs,total_engagements
471,471,1,4
1238,1238,1,4
1591,1591,2,49
2142,2142,1,2
3794,3794,3,60
4818,4818,3,21
4935,4935,3,29
5156,5156,2,284
5803,5803,1,127
null,6658,2,0


### Summary of Findings

#### Hypothesis 1: Users Who Joined eLinks a Long Time Ago Are No Longer Engaging
To test the hypothesis that older users are no longer engaging on eLinks, the relationship between the number of days on the platform and days since last engagement was assessed. Although a few clusters of users could be identified, this assessment showed that there was no linear relationship between time on platform and engagement. Based on the evidence collected, the hypothesis that users who joined eLinks a long time ago are no longer engaging can be rejected.
#### Hypothesis 2: Certain Elements of the Website Were Non-Functional, Resulting in a Decline in Engagement
To test the hypothesis that certain elements of the website weren't working, preventing users from engaging, the number of engagement activities was assessed at the weekly scale. The data showed a drop in all engagement activities in August, with the largest drop being the number of people loading the home screen. To ensure that the website was functioning normally during August, the number of daily sign up activities was also assessed. As the sign up activity looked normal during the end of August it was concluded that the website was fully functional and people were not engaging on eLinks for another reason. Based on the evidence collected, the hypothesis that certain elements of the website were non-function can be rejected.
#### Hypothesis 3: Some Users Have Switched Over to a Different Platform
Without data on user activity outside of their use of eLinks, the hypothesis that some users had switched over to a different platform was assessed by looking for patterns in the groups of users that were no longer engaging on eLinks. The engagement was plotted over time for all locations and Germany and France were found to have the largest drop in engagement. The engagement data was also examined by company, and company 1 and 2 was found to have the largest decline in engagement. This evidence suggests support for the hypothesis that users have switched to a different platform, however data on users activity on other platforms is required to confidently conclude that those users have switched to a different platform.
#### Hypothesis 4: eLinks Changed Their Engagement Techniques
To asses whether eLinks had changed their engagement techniques, the number of weekly digest emails and re-engagement emails sent were examined at the daily and weekly scale. This analysis showed a slight increase in the number of these emails sent, but no drastic changes that would explain a drop in engagement. Therefore, this hypothesis can be rejected.

In summary, there is the most evidence to support hypothesis that some users have switched over to a different platform. Without more information on the activity of users outside of their use of eLinks, it's difficult to confidently conclude that users have switched to a different platform. It is therefore recommended to conduct market analysis to assess whether the decline in engagement in Germany, France and Company and company 1 and 2 is due to a switch to a competitor.

### Recommendations

#### Recommendation 1: Conduct a Follow-up Study
As mentioned above, based on the evidence examined in this analysis it is likely that users from company 1 and 2 as well as users from Germany and France may have switched to a different networking platform. Given the non-direct evidence, it's not possible to accept this hypothesis with confidence. A follow-up study should be conducted to investigate the drop in users at companies 1 and 2 as well as users in Germany and France. This study should look for evidence that would indicate why those users have stopped engaging on eLinks and should gather additional data beyond what was explored in this analysis. The follow-up study could address the following areas: 
* Market Research
* Engagement Techniques

#### Recommendation 2: Conduct Market Research on Competitors
As part of the follow-up study, market research should be conducted to gather information on competitor networking platforms. Specific questions that the research to look to answer are: 
1. What networking platforms are leaders in the industries of companies 1 and 2? 
2. What features do those platforms have that eLinks does not currently have? 
3. What networking platforms are leaders or emerging platforms in Western Europe (Germany and France specifically)?
4. If possible, gather online survey data directly from non-engaged users to gain insight into whether they are using a different networking platform and what features they like from that platform
5. What techniques do competitors use to encourage users to engage on their platforms?

#### Recommendation 3: Utilize Effective Engagement Techniques
The Analysis of Engagement Strategies section of this analysis showed that the weekly digest emails are not an effective engagement strategy, as users who clicked through those emails did not have higher engagement rates. To ensure that any engagement strategy used is effective at getting users to interact on the eLinks, it is recommended that eLinks conducts research into different engagement techniques so they can utilize the most effective techniques. This research should aim to answer the following questions: 
1. What engagement technique leads to the highest engagement? Is it a weekly digest email? Is it biweekly highlights of the popular posts in their feed? 
2. What is the best time to send emails to users? 
3. What is the best weekday to send emails to users?

### Conclusion

In conclusion, eLinks has experienced a fairly significant drop in engagement that started at the end of July. Based on this initial analysis, there is evidence to suggest that some users have switched to a different networking platform. It is therefore recommended that eLinks conducts a follow-up study to conduct market research into competitor networking sites, gather additional information from their un-engaged users and research effective engagement techniques.